<a href="https://colab.research.google.com/github/shstreuber/Data-Mining/blob/master/Module_11_Clustering_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Clustering**

Let's face it: Most datasets don't come with classes or labels any more these days. We have to EXPLORE the data to DISCOVER the patterns within them. This process of discovery is called **"unsupervised learning"** or "clustering." Clustering allows us to segment tuples into groups based on common characteristics among their attributes. The only thing is, we don't know which groups, and often, we don't even know how many.

At the end of this module, you will be able to work with these three different types of clustering:
1. kMeans/ kMedoids
2. Hierarchical clustering
3. Density-based clustering

These three are the most popular ways of determining what happens when you don't have a class label, and you have to "make one." They explore how the data fall into "natural" groups based on their attribute values. The underlying concept for all of these investigations is, again, distance (with Euclidian distance set as the default in all three algorithms).

**Be sure to expand all the sections in this workbook, execute and observe the code, and complete all exercises!**

Start by watching the instructor video below to learn what clustering is all about:

#**What is Clustering?**

At its core, clustering involves partitioning a dataset into subsets, or clusters, such that data points within the same cluster are more similar to each other than to those in other clusters. The similarity between data points is typically measured using a distance metric, such as Euclidean distance for numerical data.

<center>
<img src = "https://blogs.sas.com/content/subconsciousmusings/files/2016/05/kmeans-clustering-plot-1.png" height = 400>
</center>

##**Why is Clustering Important?**
Clustering is widely used across various domains:

* **Data Exploration**: It helps in understanding the underlying structure of the data, uncovering hidden patterns, and gaining insights, for example in the natural sciences (e.g., grouping genes with similar expression patterns or classifying species based on genetic information)
* **Data Preprocessing**: Clusters can be used to detect anomalies, reduce dimensionality, and create more manageable subsets of data.
* **Social Network Analysis**: Identifying communities within a social network.
* **Customer Segmentation**: In marketing, clustering is used to segment customers based on their behaviors and preferences, enabling targeted marketing strategies.
* **Image and Text Analysis**: Clustering algorithms are used to group similar images or documents, aiding in tasks like image recognition and topic modeling.
* **Anomaly Detection**: Identifying outliers in financial transactions, network security, or industrial processes.

## **Getting Started with Clustering**

To get started with clustering, it's important to understand the characteristics of your data and the objectives of your analysis. Selecting the right algorithm and appropriately pre-processing your data are crucial steps in achieving meaningful clustering results.

Start by watching the instructor video below:

In [1]:
from IPython.display import IFrame  # This is just for me so I can embed videos
IFrame(src="https://www.youtube.com/embed/JKpsEFdNdQA", width=560, height=315)

Let's dive right in!

**Run the code in this module to understand how the concepts and code it together. Be sure to complete all the exercises to get a working knowledge of clustering!**

#**0. Preparation and Setup**
Similar to what we did before, we are following the basic modeling steps:

1. Exploratory Data Analysis (EDA) to see how the data is distributed--**NOTE** that we will not look for a class attribute because clustering itself is exploratory. We don't know about any classes yet!
2. Preprocess the data (remove n/a, transform data types as needed, deal with missing data)
3. Build the model--since clustering is exploratory in nature, we won't have a training or test set; we're going to explore the "natural" groupings in our data.
4. Determine the quality of the model. Since this is **NOT** classification, we will use other methods than the Confusion Matrix and the Classification Report.

We will explore our clustering algorithms on the basis of a fun dog dataset before we explore the same concept on [the famous iris dataset](https://www.kaggle.com/datasets/uciml/iris).

<center>

<img src = " https://www.researchgate.net/profile/Aydin-Ayanzadeh/publication/325384896/figure/fig3/AS:630523227549696@1527339860255/Species-of-Stanford-dog-breeds-datasets.png">
</center>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn import datasets # This is where the iris dataset will come from later
from sklearn.cluster import KMeans

# First, we import our dog dataset

dog_data = pd.read_csv('https://raw.githubusercontent.com/shstreuber/Data-Mining/master/data/dogs.csv')
dog_data = dog_data.set_index('breed')
dog_data

# **1. Exploratory Data Analysis (EDA)**
It's a fun new dataset! If you're curious what these look like, [here](https://www.akc.org/dog-breeds/) is the authoritative description of every single breed recognized in the US (with pictures)! Did you know that there are over 340 different dog breeds in the world (and that does not include the adorable mutts you find in many animal shelters)?

<div>
<center>
<img src="https://raw.githubusercontent.com/shstreuber/Data-Mining/master/images/dogs1.jpg" width="600">
</div>

Ok, back to the data (above): It looks like height and weight are in different units (inches and pounds), so in order to prepare for any calculations, we have to bring them onto the same scale. You know what this means for preprocessing: NORMALIZATION.

# **2. Preprocessing**
There's very little preprocessing we have to do with this dataset. We will use mean normalization to standardize the features to follow the Normal Distribution. That will also speed up our processing later. As we have done previously in this course, we will use the [StandardScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) from scikit-learn.

In [ ]:
# # Mean Normalization with StandardScaler

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
dog_data_array = sc.fit_transform(dog_data.values) #calculate μ & σ (fit) and apply the transformation(transform)
dog_data_array

In [ ]:
# Assign the scaled data to a DataFrame & use the index and columns arguments to keep your original indices and column names:
dog_data_norm = pd.DataFrame(dog_data_array, index=dog_data.index, columns=dog_data.columns)
dog_data_norm

In [ ]:
# Let's build a simple scatter plot
plt.scatter(dog_data_norm['weight (pounds)'], dog_data_norm['height (inches)'], alpha=0.5)
plt.title('Plot of Dog Breeds')
plt.xlabel('Normalized Weight')
plt.ylabel('Normalized Height')
plt.show()

Gazing at the scatter plot, it looks like we could group the data into three clusters. There are the 2 data points on the bottom left (Chihuahua and Yorkshire Terrier) The top right group of two (Bull Mastiff and Great Dane) and the middle group with all the other breeds.

# **3. K-Means and the Elbow Method**
K-means clustering is one of the simplest and popular unsupervised machine learning algorithms. The objective is to group similar data points together and discover underlying patterns. To achieve this objective, K-means looks for a fixed number (k) of clusters in a dataset.

Clusters are groups of data with similar characateristics. They are built around centroids. As with k Nearest Neighbor, you get to again set the value for k. But this time, **k defines the number of centroids**--that is, the imaginary or real location of the center in a cluster--you want to see in the dataset.



## **How does k Means Work?**
The KMeans function in scikit-learn is a popular clustering algorithm that partitions a dataset into a specified number of clusters, k. Here is a detailed explanation of how the KMeans function works:

###**Key Concepts of K-Means Clustering**
* **Centroids**: These are the center points of each cluster.
* **Assignment**: Each data point is assigned to the nearest centroid.
* **Update**: The centroids are recalculated based on the mean of the points assigned to them.
* **Iteration**: The assignment and update steps are repeated until the centroids no longer change significantly or a maximum number of iterations is reached.

###**Steps of the K-Means Algorithm**
1. **Initialization:**
 * Choose the number of clusters k.
 * Initialize k centroids randomly. There are different methods for initialization, such as random initialization or the k-means++ method, which selects initial cluster centers to speed up convergence.
2. **Assignment Step:**
 * Assign each data point to the nearest centroid. This step creates
k clusters of data points.
 * The distance metric commonly used is the Euclidean distance, but other metrics can be used as well.
3. **Update Step:**
 * Recalculate the centroids as the mean of all data points assigned to each cluster.
 * The new centroid of each cluster is the average of the points in the cluster.
4. **Iteration:**
 * Repeat the assignment and update steps until the centroids do not change significantly (convergence) or a pre-defined number of iterations is reached.
5. **Output:**
 * The final centroids.
 * The assignment of each data point to a cluster.

<img src = "https://assets.blog.code-specialist.com/k_means_animation_6cdd31d106.gif">

Gif from [Code Specialist](https://code-specialist.com/python/k-means-algorithm).

**Now take a look at the instructor video below:**

In [2]:
IFrame(src="https://www.youtube.com/embed/PkSnC4y5fXI", width=560, height=315)

##**Using KMeans in Scikit-learn**
The code below is a step-by-step guide to using the KMeans function in [scikit-learn.cluster](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html):

(If you want to see what kMeans used to look like when written from scratch in Python, go [here](https://benalexkeen.com/k-means-clustering-in-python/).)

### **Before We Start: Parameters and Attributes of KMeans**
You can configure (= tune) k Means with the following settings:
* **Parameters:**
 * **n_clusters**: The number of clusters to form.
 * **init**: Method for initialization, default is 'k-means++'.
 * **n_init**: Number of times the algorithm will run with different centroid seeds.
 * **max_iter**: Maximum number of iterations of the k-means algorithm.
 * **random_state**: Determines random number generation for centroid initialization.

* **Attributes:**
 * **cluster_centers_**: Coordinates of cluster centers.
 * **labels_**: Labels of each point.
 * **inertia_**: Sum of squared distances of samples to their closest cluster center.

There you have it! That's all there is to kMeans clustering--sort of.

<img src = "https://us.123rf.com/450wm/yayayoy/yayayoy2203/yayayoy220300001/183574414-happy-emoji-emoticon-showing-double-thumbs-up-like.jpg" height = 200>


##**3.1 k Means and the Dog Breeds Dataset**

In [ ]:
# Import Necessary Libraries
from sklearn.cluster import KMeans

In [ ]:
# On to clustering with kMeans. Note that the variable "labels" is an array which specifies which cluster each dog belongs to:
kmeans = KMeans(n_clusters=3, n_init = 10) # Initialize the KMeans Object and specify the number of clusters k; set n_init (number of times the algorithm will run with different centroid seeds); if needed, set max_iter (maximum number of iterations).
kmeans.fit(dog_data_norm) # Use the fit method to compute k-means clustering.
labels = kmeans.labels_
labels

My results were: array([0, 0, 0, 1, 2, 0, 0, 1, 0, 0, 2]), which indicates that the first, second, and third dogs are in cluster 0, the next one in cluster 1 and so on. That may be helpful for future tasks but is not super helpful if we are trying to visualize the data.

(**Remember** that we start with random centroids, so your results may vary slightly).
<center>
<img src = "https://t4.ftcdn.net/jpg/01/80/98/09/360_F_180980941_s5F5DsVVYie35lVTCGJHKT8ZF18weWRN.jpg" height=200>
</center>

Let's make this more "[uebersichtlich](https://en.langenscheidt.com/german-english/uebersichtlich)" (= easy to view), though (but, again, keep in mind that since the initial centroids are selected randomly it is possible that you get a different answer than I do):

In [ ]:
groups = {0: [], 1: [], 2: []} # We have 3 clusters
i = 0                                       # Initializing the row counter
for index, row in dog_data.iterrows():      # We are now going through each row in dog_data
    groups[labels[i]].append(index)         # and appending the cluster labels/ numbers to each index
    i += 1

## Now we format the output as a pretty list:

for key, value in groups.items():           # the key is the cluster number and the value is the names of the dogs
    print ('CLUSTER %i' % key)
    for breed in value:
        print("    %s" % breed)
    print('\n')

**Now let's do this with a different dataset!**

##**3.2 k Means and the Iris dataset**
Wouldn't you know it, the iris dataset is so popular that it actually comes with the scikit-learn package!!! It is formatted as an object with an array of 4 variables with one associated target variable: the species attribute. This species attribute contains the following 3 categories:

<div>
<center>
<img src="https://raw.githubusercontent.com/shstreuber/Data-Mining/master/images/iris_types.jpg" width="500">
</div>
These categories are derived from the values in the following variables:
<div>
<center>
<img src="https://raw.githubusercontent.com/shstreuber/Data-Mining/master/images/iris_measurements.png" width="300">
</div>
Petal length, petal width and sepal length and sepal width.
There you have it. The famous iris dataset.

We'll load it, and you get to do some basic EDA with the ydata_profiling package. This is the most famous dataset in Machine Learning. A [basic Google search](https://www.google.com/search?q=iris+dataset) turns up way too much information--don't read all of it; you'll get lost.



In [ ]:
from sklearn.datasets import load_iris
# Load the Iris dataset
iris = load_iris()
iris # This will show us what the data we just loaded looks like

###**Our Strategy**

We will convert the object into a dataframe and then extract the features (sepal length etc.) and the corresponding class (species).
* The class is here called the **target**.
* **HOWEVER**, for cluster analysis, we are **not** going to use the class because we want to see how the algorithm finds the **natural groupings** in the dataset.
<img src = "https://miro.medium.com/v2/resize:fit:1100/format:webp/1*TmvsQ4XaOxeb-TmKk1qgOw.png" height = 200>

(Later on, we will "cheat" and compare the algorithm results to what we know is the Ground Truth, so we call tell how good of a job our model did at grouping the data).

### **3.2.1 Exploratory Data Analysis**

##Your Turn

Use the [ydata_profiling package](https://docs.profiling.ydata.ai/latest/) to look around the dataset!


In [ ]:
# Remember that you have used the ydata_profiling packkage before!


### **3.2.2 Preprocessing**
Preprocessing is relatively easy here. First, we  need to separate the features from the labels. Remember that we don't want the labels as any input to kMeans because we are asking kMeans to discover natural clusters based on unlabelled data.

In [ ]:
# iris_data contains the features (sepal length, sepal width, petal length, petal width--all in centimeters)
# iris_target contains the class labels, which we are setting below (0='setosa', 1='versicolor', 2='virginica')
iris_data = iris.data
iris_target = iris.target

Next, we need to ensure that iris_target_name is correctly defined and contains mappings from all indices in iris.target to their respective names.

In [ ]:
# Defining iris_target_name as a dictionary mapping indices to names
iris_target_name = {
    0: 'Setosa',
    1: 'Versicolor',
    2: 'Virginica'
}

### **3.2.3 Building k Means**
Now that we have loaded and processed our data, we will go ahead and fit a K-means model to the data. We will use 3 of the 4 features available (sepal length, sepal width, petal length). **This will make it easier for us to visualize the results of the model in a 3D graph.**

Because we know that the class attribute has three levels (virginica, setosa, versicolor), we decide to set k=3.

In [ ]:
# Build kmeans and fixing the random state so we are always getting the same results. Setting n_init to the default

kmeans_3 = KMeans(n_clusters=3, random_state=42, n_init = 10, verbose = 0)
# kmeans_3 = KMeans(n_clusters=3, random_state=42, n_init = 10, verbose = 1) # uncomment if you want to see how it cycles through the centroid assignment

In [ ]:
# Fit the K-Means model to the data

kmeans_iris = kmeans_3.fit(iris_data[:, :3])  # Use columns 0, 1, and 2 for clustering
kmeans_labels = kmeans_iris.labels_
centroids = kmeans_iris.cluster_centers_

### **3.2.4 Visualizing the Clusters**

In [ ]:
# Create a 3D plot
fig = plt.figure(1, figsize=(8, 6))
ax = fig.add_subplot(111, projection='3d', elev=48, azim=134)

# Plot data points
scatter = ax.scatter(iris_data[:, 0], iris_data[:, 1], iris_data[:, 2], c=kmeans_labels, s=50, cmap='viridis')

# Plot cluster centroids in red for better visibility
ax.scatter(centroids[:, 0], centroids[:, 1], centroids[:, 2], c='red', s=200, alpha=0.5)

# Annotate clusters
for name, label in [('Setosa', 0), ('Versicolor', 1), ('Virginica', 2)]:
    ax.text3D(iris_data[kmeans_labels == label, 0].mean(),
              iris_data[kmeans_labels == label, 1].mean() + 1.5,
              iris_data[kmeans_labels == label, 2].mean(), name,
              horizontalalignment='center',
              bbox=dict(alpha=.5, edgecolor='w', facecolor='w'))

# Set labels and title
ax.set_xlabel('Sepal length (cm)')
ax.set_ylabel('Sepal width (cm)')
ax.set_zlabel('Petal length (cm)')
ax.set_title('kMeans Output with k=3')
plt.colorbar(scatter, ax=ax, shrink=0.5, aspect=5)  # Add color bar to indicate cluster colors
plt.show()


### **3.2.5 Quality Check!**
**Let's cheat!**

We already know that we should have discovered three clusters because the original dataset contains three types of irises in the species class. So, let's compare the clusters that we have discovered in kmeans with the actual iris_target labels in the species attribute.

We will use a confusion matrix to do this!

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Create confusion matrix
cm = confusion_matrix(iris_target, kmeans_labels)

# Create ConfusionMatrixDisplay with labeled classes
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=iris.target_names).plot()

###Your Turn
What does the confusion matrix tell you about the results of our kmeans cluster detection? Write your answer below.

### **3.2.6 The BIG Question**
The next question is: How big should k be? Let's experiment!

In [ ]:
# Now we plot the output of our kMeans analysis, for k=3 and k=8

# Load a fresh Iris dataset
iris = load_iris()
X = iris.data  # Features
y = iris.target  # Target

# Define the kMeans estimators with different number of clusters
estimators = [('k_means_iris_8', KMeans(n_clusters=8, random_state=42, n_init=10, verbose=0)),
              ('k_means_iris_3', KMeans(n_clusters=3, random_state=42, n_init=10, verbose=0))]

# Set up figure parameters
fig = plt.figure(figsize=(10, 4))
fig.subplots_adjust(left=0.02, right=0.98, bottom=0.05, top=0.9)
plt.suptitle('Comparison of kMeans Clustering Results', fontsize=16)

# Iterate over estimators to create plots
for idx, (name, est) in enumerate(estimators, 1):
    ax = fig.add_subplot(1, 2, idx, projection='3d', elev=48, azim=134)

    # Fit the kMeans estimator and predict clusters
    est.fit(X)
    labels = est.labels_

    # Scatter plot of the data points colored by cluster labels
    ax.scatter(X[:, 3], X[:, 0], X[:, 2], c=labels.astype(float), edgecolor='k')

    # Set axis labels and title
    ax.set_xlabel('Petal width')
    ax.set_ylabel('Sepal length')
    ax.set_zlabel('Petal length')
    ax.set_title('{} clusters'.format(name.split('_')[-1]), fontsize=14)

    # Hide tick labels
    ax.xaxis.set_ticklabels([])
    ax.yaxis.set_ticklabels([])
    ax.zaxis.set_ticklabels([])

plt.show()


8 clusters does not show any improvement. So, we're back to our initial 3. Let's see how well the clusters are defined:

In [ ]:
# Calculate and print value counts
print(pd.Series([(iris_target_name.get(i, 'Unknown'), kmeans_iris.labels_[i]) for i in range(len(iris.target))]).value_counts())

Not too shabby! As you can see, we have a header row with the actual names that are included in the results. Will they make a big difference? No.

Want to see the assignment for each row? Then execute the cell below. This output is LONG!

In [ ]:
# Print each pair of (target name, cluster label)--usually for debugging purposes
for i in range(len(iris.target)):
    target_name = iris_target_name.get(i, 'Unknown')  # Handle missing keys gracefully
    cluster_label = kmeans_iris.labels_[i]
    print(f"Index {i}: Target Name = {target_name}, Cluster Label = {cluster_label}")

As the above visualizations show, our simple model with k=3 has done a sort-of decent task of classifying the datapoints into separate clusters. You can see that the two plots resemble each other. When compared with actual classes, the model clustered together some datapoints belonging to different classes. Although the predictions aren’t perfect, they come close. That’s a win for the algorithm.

## **3.3 Elbow method for finding optimal number of clusters**
In unsupervised learning, you rarely get an output that’s 100 percent accurate because real-world data is just not that simple.
Apart from simplicity of the dataset, the reason why the above model performed so well was because we knew beforehand the optimal numbers of clusters, i.e. 3. Out in the wild, you won’t know how many clusters to choose (or any initialization parameter for other clustering algorithms). Such parameter-tuning is critical.

One of the methods for determining the optimal number of clusters for the K-Means algorithm is the elbow method.

<img src = "https://media.licdn.com/dms/image/D4D12AQF-yYtbzPvNFg/article-cover_image-shrink_600_2000/0/1682277078758?e=2147483647&v=beta&t=VhzheKDjy7bEcsYyrjql3NQAUcTaMBCTzhZWSVVSeNg">

The idea of the elbow method is to run k-means clustering on the dataset for a range of values of k (say, k from 1 to 10), and for each value of k calculate the [sum of squared errors](https://www.analyticsvidhya.com/blog/2021/01/in-depth-intuition-of-k-means-clustering-algorithm-in-machine-learning/). Then, plot a line chart of the SSE for each value of k. If the line chart looks like an arm, then the "elbow" on the arm is the value of k that is the best.

The idea is that we want a small SSE, but that the SSE tends to decrease toward 0 as we increase k (the SSE is 0 when k is equal to the number of data points in the dataset, because then each data point is its own cluster, and there is no error between it and the center of its cluster). So our goal is to choose a small value of k that still has a low SSE, and the elbow usually represents where we start to have diminishing returns by increasing k.

**NOTE**: Until now we have worked with our data as a NumPy array. In order to use elbow plotting elegantly, we need to convert the NumPy Array to a Pandas DataFrame. This conversion allows for column-based indexing using column names. Then we can use the DataFrame Column Names, which now allows the elbow_plot function to access the data using the column names provided.

**Be sure to execute the code below to see how the elbow builds!**

In [ ]:
# Elbow method function
def elbow_plot(data, maxK=10, seed_centroids=None):
    """
    parameters:
    - data: pandas DataFrame (data to be fitted)
    - maxK (default = 10): integer (maximum number of clusters with which to run k-means)
    - seed_centroids (default = None): float (initial value of centroids for k-means)
    """
    sse = {}
    for k in range(1, maxK):
        print("k:", k)
        if seed_centroids is not None:
            seeds = seed_centroids.head(k)
            kmeans = KMeans(n_clusters=k, max_iter=500, n_init=100, random_state=0, init=np.reshape(seeds, (k, 1))).fit(data)
            data["clusters"] = kmeans.labels_
        else:
            kmeans = KMeans(n_clusters=k, max_iter=300, n_init=100, random_state=0).fit(data)
            data["clusters"] = kmeans.labels_
        # Inertia: Sum of distances of samples to their closest cluster center
        sse[k] = kmeans.inertia_
    plt.figure()
    plt.plot(list(sse.keys()), list(sse.values()))
    plt.xlabel("Number of clusters")
    plt.ylabel("SSE")
    plt.title("Elbow Method for Optimal k")
    plt.show()
    return

# Use the function with the first three features of the Iris dataset
elbow_plot(iris_data[['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)']], maxK=10)


As we can see from the above plot, 2 or 3 look like a good choice for number of clusters--2 because that's where the really obvious break occurs, and 3 because the slope of the curve becomes pretty linear after 3.

**Wait ... 2? Or 3?**

<center>
<img src = "https://1000logos.net/wp-content/uploads/2024/01/Emoji-Confused.png" height = 200>
</center>

If you scroll up to the graphics, you'll be able to tell why: Look how the virginica and the versicolor "clusters" are intertwined! We can only separate them if we really know they are there, but they are still discoverable. That's why the elbow curve changes direction at 2 clusters, and becomes linear after 3. The trick is to ***look for*** the greatest difference in slope but to ***pick*** the point **AT WHICH** which the slope decreases in a linear fashion. So, the "more correct" answer is 3.
<center>
<img src = "https://gifdb.com/images/high/eye-roll-emoji-face-palm-annoyed-reaction-idffmtkferlzl0ej.gif" height=300>
</center>

As you can see, the elbow criterion doesn't always give you very clear answers. In cases like this, we might try a different method for determining the optimal k, such as [computing silhouette scores](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.silhouette_score.html), or we might even reevaluate whether clustering is the right thing to do on our data.

## Your Turn
**Exercise 1: kMeans for the dog_data**


Make an elbow plot for the dog data above and then re-run the kMeans analysis with the optimal k. Plot the results.

In [ ]:
# Start with the elbow plot in this cell

In [ ]:
# Now run k Means with the optimal k you have just determined.

# **4. Hierarchical Clustering**
For the next approach **we don’t specify how many clusters to make**. Instead the algorithm starts with each instance in its own cluster. At each iteration of the algorithm it combines the two most similar clusters into one. It repeatedly does this until there is only one cluster. This is called hierarchical clustering and its name makes sense. The running of the algorithm results in one cluster, which consists of two sub-clusters. Each of those two sub-clusters in turn, consist of 2 sub-sub clusters and so on.

We have 3 types of clusters:
1. **Single-Link**: In single-linkage clustering we define the distance between two clusters as the shortest
distance between any member of one cluster to any member of the other.
2. **Complete-Link**: In complete-linkage clustering we define the distance between two clusters as the greatest
distance between any member of one cluster to any member of the other.
3. **Average-Link**: In average-linkage clustering we define the distance between two clusters as the average
distance between any member of one cluster to any member of the other.

<img src = 'https://raw.githubusercontent.com/shstreuber/Data-Mining/master/images/The-three-linkage-types-of-hierarchical-clustering-single-link-complete-link-and.png'>

For this purpose, we use the Euclidian distance again.

Watch the video below and then read more about clustering [here](https://scikit-learn.org/stable/modules/clustering.html#hierarchical-clustering).

In [3]:
IFrame(src="https://www.youtube.com/embed/EUQY3hL38cw", width=560, height=315)

Hierarchical clustering can be done in two main ways: **AGGLOMERATIVE Clustering** and **DIVISVE Clustering**. These methods differ in how they build the hierarchy of clusters. Here's a simple explanation of the difference between them:

1. **Agglomerative Clustering** is a bottom-up approach. Here's how it works:

 * **Start with Individual Items**: Initially, each item is its own cluster. For example, if you have five items (A, B, C, D, E), you start with five clusters: [A], [B], [C], [D], [E].

 * **Find the Closest Pair**: Look at all the clusters and find the two that are most similar (closest to each other).

 * **Merge the Closest Pair**: Combine these two clusters into a single cluster. Now you have one less cluster. Repeat this process. For instance, if [A] and [B] are the closest, merge them into [A, B].

 * **Repeat Until One Cluster Remains**: Continue merging the closest pairs of clusters until there is only one cluster left that contains all the items.

This process builds a hierarchy from the **bottom up**, creating a dendrogram (tree) that shows how the items were grouped together step by step. It is more frequently used because of its self-explanatory implementation.

2. **Divisive Clustering** is a **top-down** approach, starting with one big cluster and breaking it down into smaller and smaller clusters, eventually reaching individual items. It is less common but can be useful in specific cases where you need to split large heterogeneous clusters.

## **4.1 Agglomerative Clustering**
We are using the AgglomerativeClustering() function [link text](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.AgglomerativeClustering.html) from scikit-learn.cluster.

The AgglomerativeClustering object performs a hierarchical clustering using a bottom up approach: each observation starts in its own cluster, and clusters are successively merged together. The linkage criteria (see above) determines the metric used for the merge strategy.

In [ ]:
from sklearn.cluster import AgglomerativeClustering

dog_data = pd.read_csv('https://raw.githubusercontent.com/shstreuber/Data-Mining/master/data/dogs.csv')
dog_data = dog_data.set_index('breed')
dog_data

In [ ]:
clusterer = AgglomerativeClustering(affinity='euclidean', linkage='ward')
clusterer.fit(dog_data_norm)

In [ ]:
clusterer.labels_

So here the first dog breed, Border Collie, belongs to cluster 0. Keep in mind that in kMeans there is a random element--the selection of the initial centroids, but **in hierarchical clustering there is no randomness** so you should get the exact same answer I have here.

## **4.2 Hierarchical Clustering and Dendrograms**
Remember that the hierarchical clustering algorithm constructs a tree - specifically, a dendrogram. To view that requires some imagination. You can print dendrograms in different ways. A very plain and ugly way is here:

In [ ]:
import itertools
ii = itertools.count(dog_data_norm.shape[0])
[{'node_id': next(ii), 'left': x[0], 'right':x[1]} for x in clusterer.children_]

The first line {'left': 0, 'node_id': 11, 'right': 8} reads like this: "We combine cluster 0 Border Collie with cluster 8 Portuguese Water Dog to create Cluster 11." The next line says we combine 4 Chihuahua with 10 Yorkshire Terrier to create cluster 12. Let's associate index numbers with the dog breed names so that structure is easier to read:

In [ ]:
dog_names = pd.DataFrame({'breeds': dog_data_norm.index.values})
dog_names

That makes it easier to interpret lines like:

{'left': 1, 'node_id': 14, 'right': 2},
We are combining 1 Boston Terrier and 2 Brittany Spaniel.![dendro[1].png]('sxkaXYgY2xhc3M9IkJ0bkdyb3VwIG10LTEgbXQtbWQtMCBtbC0yIj4KICAgICAgICAgICAgICAgIDxidXR0b24gY2xhc3M9ImJ0biBidG4tb3V0bGluZSBCdG5Hcm91cC1pdGVtIGpzLWFjY2VwdC1hbmFseXRpY3MtY29va2llcyIgdHlwZT0iYnV0dG9uIj5BY2NlcHQ8L2J1dHRvbj4KICAgICAgICAgICAgICAgIDxidXR0b24gY2xhc3M9ImJ0biBidG4tb3V0bGluZSBCdG5Hcm91cC1pdGVtIGpzLXJlamVjdC1hbmFseXRpY3MtY29va2llcyIgdHlwZT0iYnV0dG9uIj5SZWplY3Q8L2J1dHRvbj4KICAgICAgICAgICAgICA8L2Rpdj4KICAgICAgICAgICAgPC9kaXY+CiAgICAgICAgICA8L2Rpdj4KCiAgICAgICAgICA8ZGl2IGNsYXNzPSJ0ZXh0LXJpZ2h0IHB5LTMgYm9yZGVyLXRvcCI+CiAgICAgICAgICAgIDxidXR0b24gY2xhc3M9ImJ0biBidG4tcHJpbWFyeSBqcy1zYXZlLWNvb2tpZS1wcmVmZXJlbmNlcyIgdHlwZT0iYnV0dG9uIiBkaXNhYmxlZD5TYXZlIHByZWZlcmVuY2VzPC9idXR0b24+CiAgICAgICAgICA8L2Rpdj4KICAgICAgICA8L2Rpdj4KPC9kaXY+PC9kaXY+ICA8L2Rpdj4KPC9kaXY+CgoKICA8L2JvZHk+CjwvaHRtbD4KCg==)

That's what we get when we draw this.



That still isn't really pretty. Let's plot this as a dendrogram (with plotly's figure factory)!

In [ ]:
import plotly.figure_factory as ff
fig = ff.create_dendrogram(dog_data, labels=dog_data.index)
fig.update_layout(width=800, height=500)
fig.show()

We can also plot with pandas, which saves us one line of code:

In [ ]:
from plotly.figure_factory import create_dendrogram
fig = create_dendrogram(dog_data, labels=dog_data.index)
fig.show()

## Your Turn
**Exercise 2: Run a hierarchical analysis for the Iris dataset**

You've seen the analysis for the dogs above. Now run the same thing for the iris dataset.

In [ ]:
# Use this field for your analysis or the iris dataset


# **5. Density-Based Clustering (DBSCAN)**
Dansity-Based clustering finds core samples of high density and expands clusters from them. In other words, it views clusters as areas of high density separated by areas of low density. Good for data which contains clusters of similar density.Here, we are working with neighbors within a radius!

We are using the [DBSCAN algorithm](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html) from the scikit-learn.cluster family. Because DBSCAN really requires visualizzation, we'll start with a dataset that allows us to visualize the output the best: The [make_moons dataset](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_moons.html).

Read more about how DBSCAN works [here](https://towardsdatascience.com/how-dbscan-works-and-why-should-i-use-it-443b4a191c80) and watch the video below:

In [4]:
IFrame(src="https://www.youtube.com/embed/RDZUdRSDOok", width=560, height=315)

Let's get started!

In [ ]:
## import data
from sklearn.datasets import make_moons
from sklearn.datasets import load_iris # reloading iris cleanly
from sklearn import datasets

## import DBSCAN model
from sklearn.cluster import DBSCAN
from sklearn import metrics

plt.style.use('ggplot')
%matplotlib inline

##**5.1 Example 1: make_moons dataset**
This is just a small dataset that help visualize density-based clustering. It has no meaning other than the shape you see.

In [ ]:
# import the data from sklearn.datasets.make_moons
X_1, labels_true = make_moons(n_samples=200, noise=0.1, random_state=19)
print(X_1[:10,])

##**5.1.1 Setting up our DBSCAN model**

Note that the DBSCAN model requires two parameters, eps and min_samples.

* eps is a radius which defines neighborhood for a data point.
* min_samples (or minPts) is the minimum number of points required to form a dense region within a radius eps.

In [ ]:
model_1 = DBSCAN(eps=0.25, min_samples=12).fit(X_1)
print(model_1) # model set-ups and parameters

In [ ]:
## Another way to see parameters and definition of the DBSCAN model
model_1.get_params

##**5.1.2 Reviewing the labels**
We can now take a look at the cluster labels that our model_1 has assigned.

In [ ]:
labels_1=model_1.labels_
labels_1

In [ ]:
# Alternative way to access the cluster labels (the same as above)
model_1.fit_predict(X_1)

## **5.1.3 Plotting the clusters**
Visualization is really important with these kinds of clusters.

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
ax.scatter(X_1[:, 0], X_1[:, 1], c=model_1.labels_, s=140, alpha=0.9, cmap=plt.cm.Set1)

fig.show()

We can also get the answer about the detected clusters like this:

In [ ]:
# Number of clusters in labels, ignoring noise points
n_clusters_ = len(set(labels_1)) - (1 if -1 in labels_1 else 0)
print('Estimated number of clusters: %d' % n_clusters_)

If there is noise in this dataset, we can also identify the noise points:

In [ ]:
print("Number of Noise Points: ",sum(model_1.labels_==-1)," (",len(model_1.labels_),")",sep='')

## **5.2 DBSCAN and the iris dataset**
Here is an analysis of the iris dataset again.

In [ ]:
# Import the Iris dataset from scikit-learn
iris = datasets.load_iris()
X = iris.data
y = iris.target
target_names = iris.target_names

Let's convert it to a dataframe again.

In [ ]:
df = pd.DataFrame(X, columns=iris.feature_names)
df.head()

Let's see what a basic scatterplot shows us.

In [ ]:
## Show only two features 'Sepal length' and 'petal length' for different types of flowers (target labels)
plt.figure(2, figsize=(10, 6))
colors = ['orange', 'red', 'gray']


for color, i, target_name in zip(colors, [0, 1, 2], target_names):
    plt.scatter(X[y == i, 1], X[y == i, 3], color=color, alpha=.8,
                label=target_name, s=150, edgecolor='k',marker='D')
plt.legend(loc='best', shadow=False, scatterpoints=1,fontsize=20)
plt.xlabel('sepal length', fontsize=30)
plt.ylabel('petal length', fontsize=30)
plt.show()

## **5.2.1 Setting up the model**

In [ ]:
model = DBSCAN(eps=0.8, min_samples=6).fit(X)
labels=model.labels_
labels

Getting the number of clusters.

In [ ]:
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_clusters_

## **5.2.2 Plotting the clusters**

In [ ]:
plt.figure(2, figsize=(10, 6))
plt.scatter(X[:, 1], X[:, 3], c=model.labels_, s=150,  cmap=plt.cm.Set1,alpha=.8, edgecolor='k',marker='D')
#plt.scatter(X[:, 1], X[:, 3], c=y, s=150, cmap=plt.cm.Set1, edgecolor='k', marker='D')   #target
plt.xlabel('Sepal length', fontsize=30)
plt.ylabel('petal length', fontsize=30)
plt.title('Estimated number of clusters: %d' % n_clusters_, fontsize=20)
plt.show()

## **5.2.3 Evaluating the model: Completeness & homogeneity scores**

These scores tell you how well the clustering algorithm performed.
* Perfect homogeneity score=1, perfect completeness score =1: your clustering matches your classes perfectly.

* A clustering result satisfies homogeneity if all of its clusters contain only data points which are members of a single class.

* A clustering result satisfies completeness if all the data points that are members of a given class are elements of the same cluster.

In [ ]:
# Examples for perfect scores
# Perfect labelings are complete and homogeneous:
print("Completeness: %0.1f" %  metrics.completeness_score([0, 0, 1, 1], [1, 1, 0, 0]))

In [ ]:
# completeness: for a given class, are all its points placed in one cluster?
print("Completeness: %0.1f" %  metrics.completeness_score([0, 0, 1, 1], [0, 0, 0, 0]))

In [ ]:
# Homogeneity: for a given cluster, do all the points in it come from the same class?
print("Homogeneity: %0.1f" %  metrics.homogeneity_score([0, 0, 1, 1], [0, 0, 0, 0]))

And now back to our iris dataset!

In [ ]:
model_2 = DBSCAN(eps=0.8, min_samples=6).fit(X)
labels=model_2.labels_
labels_true = iris.target

print("Completeness: %0.2f" % metrics.completeness_score(labels_true, labels))
print("Homogeneity: %0.2f" % metrics.homogeneity_score(labels_true, labels))

##Your Turn
**Exercise 3: DBSCAN and Dog Breeds**

Apply DBSCAN to the dog breeds dataset using the code fields below. Create a visualization.

In [ ]:
# Build your DBSCAN algorithm here

In [ ]:
# Create your visualization here

In the field below, describe how the output from this algorithm differs from what you have seen with k Means and Hierarchical Clustering.



Which of the three clustering approaches is best for the dog breeds dataset? Which one works best for the iris dataset? Answer these two questions in the field below.

#**Congratulations! You're done with this module!**

<div>
<center>
<img src="https://raw.githubusercontent.com/shstreuber/Data-Mining/master/images/dog_smile.jpg" width="500">
</div>